In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU in use:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. Using CPU.")

CUDA is NOT available. Using CPU.


In [ ]:
import torch
torch.cuda.empty_cache()

### Loading the model and transcribing the sample_audio

In [18]:
# c2t model loading example
import whisper_s2t
from whisper_s2t.backends.ctranslate2.model import BEST_ASR_CONFIG


# model_kwargs = {
#     'device': 'cpu',  # Use 'cpu' if you don't have a GPU
#     'compute_type': 'int8', # Note int8 is only supported for CTranslate2 backend, for others only float16 is supported for lower precision.
#     'asr_options': BEST_ASR_CONFIG
# }



ct2_model = whisper_s2t.load_model(
    model_identifier="large-v2", 
    backend='CTranslate2', 
    compute_type='int8',  # Best for CUDA
    device='cpu',
    asr_options=BEST_ASR_CONFIG,
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
import importlib
import split_and_preprocess
importlib.reload(split_and_preprocess)
from split_and_preprocess import process_chunks


output_path = "output.wav"
audio_path = "test.wav"

# Process and merge audio chunks
merged_audio, sr = process_chunks(audio_path, output_path, chunk_duration=30)

Loaded audio shape: (273600,), Sample Rate: 48000
Estimated speech rate: 4.39 onsets/sec
Speech rate is normal.
Original length: 273600, Processed length: 273600
RMS dB level: -17.59 dBFS
Saved merged processed audio to output.wav


In [17]:
lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']


files = [output_path]
    
ct2_out = ct2_model.transcribe_with_vad(
        files,
        lang_codes=lang_codes,
        tasks=tasks,
        initial_prompts=initial_prompts,
        batch_size=16
    )

    # cleaned_segments = [
    #     seg.text for seg in segments
    #     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
    # ]
    # cleaned_transcript = " ".join(cleaned_segments)

print("c2t English translation:")
print(ct2_out)   # English text

Transcribing: 100%|██████████| 100/100 [00:51<00:00,  1.93it/s]

c2t English translation:
[[{'text': 'I wish you all the best.', 'avg_logprob': -0.718155674636364, 'no_speech_prob': 0.2597784101963043, 'start_time': np.float64(0.9), 'end_time': np.float64(5.12)}]]
